In [11]:
from datetime import datetime, timedelta
import pandas as pd
import copy
import numpy as np
from geneticalgorithm2 import geneticalgorithm2 as ga
import math
import random
import numpy as np
import time

from dateutil import parser

from robot_events_api import *

In [2]:
    



def input_els(file):
    test = pd.read_excel(file,engine='calamine')
    test['Datetime'] = test['Datetime'].apply(lambda x: x[0:16])
    test['Datetime'] = test['Datetime'].apply(lambda x: datetime.strptime(x, '%d-%b-%Y %H:%M')+ timedelta(hours=18))
    test['Match_type'] = test['Match'].apply(lambda x: x.split(" #")[0])
    test = test[~test['Match_type'].str.contains("Match")]
    return test

f2 = input_els('23_es.xls')

f2

,Datetime,Match,Red Team 1,Blue Team 1,Red Score,Blue Score,Match_type
0,2023-12-09 10:29:00,Practice #1,14888B,2223R,42,42.0,Practice
1,2023-12-09 10:30:00,Practice #2,3168B,3168H,82,82.0,Practice
2,2023-12-09 10:34:00,Practice #3,22020N,2223K,85,85.0,Practice
3,2023-12-09 10:36:00,Practice #4,2223F,14888C,76,76.0,Practice
4,2023-12-09 10:40:00,Practice #5,2223M,44127B,53,NaN,Practice
...,...,...,...,...,...,...,...
166,2023-12-10 11:46:00,TeamWork #151,2223K,2223J,80,80.0,TeamWork
167,2023-12-10 11:49:00,TeamWork #152,2223M,29010B,68,68.0,TeamWork
168,2023-12-10 11:52:00,TeamWork #153,14888B,2223B,78,78.0,TeamWork
169,2023-12-10 11:54:00,TeamWork #154,21997A,3168H,78,78.0,TeamWork


In [16]:
f2.iloc[0]['Datetime']

Timestamp('2023-12-09 10:29:00')

In [19]:
def get_schedule(Division):   
    if Division == 'Middle school':
        response_q = requests.get("https://www.robotevents.com/api/v2/events/56495/divisions/1/matches?&per_page=250", headers=headers)
    else:
        response_q = requests.get("https://www.robotevents.com/api/v2/events/56493/divisions/1/matches?&per_page=250", headers=headers)


    data_q =response_q.json()



    #print(data_q)

    # sd_list = []
    # sp_list = []


    q_list = []
    for x in data_q['data']:
        #print(x['round'],x['matchnum'],x['scheduled'],x['name'],'\n',x['alliances'][0]['score'],'\n',x['alliances'][1])
        if x['round'] != 15:
            #q_list.append([x['round'],x['matchnum'],x['scheduled'],x['name'],x['alliances'][0]['teams'][0]['team']['name'],x['alliances'][1]['teams'][0]['team']['name']])
            if x['round'] == 0:
                type_M = 'Practice'
            else:
                type_M = 'TeamWork'

            q_list.append([x['scheduled'],
                           x['name'],
                           x['alliances'][0]['teams'][0]['team']['name'],
                           x['alliances'][1]['teams'][0]['team']['name'],
                           x['alliances'][0]['score'],
                           x['alliances'][1]['score'],
                           type_M                         
                           ])
            #print(x['round'],x['matchnum'])
    df=pd.DataFrame(q_list,columns =['Datetime','match','Red Team 1','Blue Team 1','Red Score','Blue Score','Match_type'])
    df['Datetime'] = df['Datetime'].apply(lambda x: parser.parse(x).replace(tzinfo=None)+ timedelta(hours=18))
    #df['Datetime'] = df['Datetime'].apply(lambda x: x.)

#parser.parse(ds)
    return df

a= get_schedule('Middle school')

a

,Datetime,match,Red Team 1,Blue Team 1,Red Score,Blue Score,Match_type
0,2024-12-07 10:30:00,Practice #1,3204M,22020N,47,47,TeamWork
1,2024-12-07 10:33:00,Practice #2,14888D,3204A,27,27,TeamWork
2,2024-12-07 10:36:00,Practice #3,3168B,48495M,36,36,TeamWork
3,2024-12-07 10:39:00,Practice #4,22020W,35460A,48,48,TeamWork
4,2024-12-07 10:42:00,Practice #5,48495A,88397F,83,83,TeamWork
...,...,...,...,...,...,...,...
187,2024-12-08 12:13:48,TeamWork #172,6936B,48495M,47,47,TeamWork
188,2024-12-08 12:16:21,TeamWork #173,22020N,48495C,134,0,TeamWork
189,2024-12-08 12:18:54,TeamWork #174,48495F,4714A,57,57,TeamWork
190,2024-12-08 12:21:27,TeamWork #175,8757V,48495E,20,20,TeamWork


In [15]:
a.iloc[0]['Datetime']

Timestamp('2024-12-06 16:30:00-0500', tz='tzoffset(None, -18000)')